# Multi-Armed Bandits - Solution Methods

In [ ]:
import numpy as np
import pandas as pd
import cufflinks as cf
import plotly.offline

cf.go_offline()
cf.set_config_file(world_readable=True, theme="white")

from MAB.bandits import GaussianBanditGame, GaussianBandit, BernoulliBandit

## 1. A Gaussian bandit game

In [ ]:
slotA = GaussianBandit(5, 3)
slotB = GaussianBandit(6, 2)
slotC = GaussianBandit(1, 5)

In [ ]:
game = GaussianBanditGame([slotA, slotB, slotC])

In [ ]:
game.user_play()

## 2. Online Advertising Example

In [ ]:
adA = BernoulliBandit(0.004)
adB = BernoulliBandit(0.016)
adC = BernoulliBandit(0.02)
adD = BernoulliBandit(0.028)
adE = BernoulliBandit(0.031)
ads = [adA, adB, adC, adD, adE]

### 2.1 Strategy 1: A/B/n testing

This is an exploration strategy used to determine which action should be taken by directly comparing actions. An experiment is run and at the end of the experiment, the results are compared for each action.

This can be seen as a baseline strategy for solving the problem.

Suppose you select an action $a$ for the $i$th time, for which you get reward $R_i$. The average reward observed prior to the $n^{th}$ selection is $$Q_n \equiv \frac{R_1 + ... + R_n}{n-1}$$. We can define an update rule by factoring out the $R_n$ term and multiplying by $(n-1)/(n-1)$. This gives us
$$Q_{n+1} = Q_n + \frac{1}{n}(R_n - Q_n)$$

This tells us that to update the expected reward at the $(n+1)^{th}$ action, we just need to add the deviation of the reward from the expected value, divided by the total number of actions taken. As we make more observations, our corrections to the expected reward will get smaller and smaller.

An interesting thing to note is that this could be a limitation if the environment changes with time, in which case we may want more recent observations to have the same or more importance.

In [ ]:
n_test = 10_000
n_prod = 90_000
n_ads = len(ads)
Q = np.zeros(n_ads)  # Q, action values
N = np.zeros(n_ads)  # N, total impressions
total_reward = 0
avg_rewards = []  # save average rewards over time

In [ ]:
# Each turn, randomly select an action to take
for i in range(n_test):
    ad_chosen = np.random.randint(n_ads)
    R = ads[ad_chosen].pull_lever()  # observe reward
    N[ad_chosen] += 1
    Q[ad_chosen] += (1 / N[ad_chosen]) * (R - Q[ad_chosen])
    total_reward += R
    avg_reward_so_far = total_reward / (i + 1)
    avg_rewards.append(avg_reward_so_far)

In [ ]:
best_ad_index = np.argmax(Q)
print(f"The best performing ad is ad {['A', 'B', 'C', 'D', 'E'][best_ad_index]}")

In [ ]:
# Make a choice as to which ad was best for the test period and use that in production
ad_chosen = best_ad_index
for i in range(n_prod):
    R = ads[ad_chosen].pull_lever()
    total_reward += R
    avg_reward_so_far = total_reward / (n_test + i + 1)
    avg_rewards.append(avg_reward_so_far)

In [ ]:
df_rewards = pd.DataFrame(avg_rewards, columns=["A/B/n"])

df_rewards.iplot(
    title=f"A/B/n Test Avg. Reward: {avg_reward_so_far:.4f}",
    xTitle="Impressions",
    yTitle="Avg. Reward",
)

Using this strategy, we can see that after the exploration phase ends, the average reward consistently grows until it plateaus around the average for campaign E.

#### Issues with A/B/n testing

* It is inefficient with the samples and does not modify the experiment dynamically by learning from observations. It doesn't take advantage of any information to cull non-promising campaigns early, for example.
* It is unable to correct a decision once it's made. If during the test period the wrong "best" campaign is selected, then it is fixed for the production period. It cannot adapt.
* It cannot adapt to changes in a dynamic environment, especially so for non stationary environments.
* The length of the test period is a hyperparameter that has a significant effect on performance and on cost



### 2.2 Strategy 2: $\epsilon$-Greedy Actions

The $\epsilon$-greedy approach corrects the static nature of A/B/n testing by allowing for continuous exploration.

In essence, the user should always take the greedy action that gives the best reward with probability $1 - \epsilon$. However, with probability $\epsilon$ it should take a random action that could be sub-optimal. Typically, the value of $\epsilon$ is kept small to exploit the knowledge developed.

In [ ]:
eps = 0.01
n_prod = 100_000
n_ads = len(ads)
Q = np.zeros(n_ads)
N = np.zeros(n_ads)
total_reward = 0
avg_rewards = []

In [ ]:
# ad_chosen = np.random.randint(n_ads)
# for i in range(n_prod):
#     R = ads[ad_chosen].pull_lever()
#     N[ad_chosen] += 1
#     Q[ad_chosen] += (1 / N[ad_chosen]) * (R - Q[ad_chosen])
#     total_reward += R
#     avg_reward_so_far = total_reward / (i + 1)
#     avg_rewards.append(avg_reward_so_far)

#     if np.random.uniform() <= eps:
#         ad_chosen = np.random.randint(n_ads)
#     else:
#         ad_chosen = np.argmax(Q)

# df_rewards[f"e-greedy: {eps}"] = avg_rewards

In [ ]:
avg_rewards = []
for eps in [0.01, 0.05, 0.1, 0.2]:
    Q = np.zeros(n_ads)
    N = np.zeros(n_ads)
    total_reward = 0
    ad_chosen = np.random.randint(n_ads)
    for i in range(n_prod):
        R = ads[ad_chosen].pull_lever()
        N[ad_chosen] += 1
        Q[ad_chosen] += (1 / N[ad_chosen]) * (R - Q[ad_chosen])
        total_reward += R
        avg_reward_so_far = total_reward / (i + 1)
        avg_rewards.append(avg_reward_so_far)

        if np.random.uniform() <= eps:
            ad_chosen = np.random.randint(n_ads)
        else:
            ad_chosen = np.argmax(Q)

    df_rewards[f"e-greedy: {eps}"] = avg_rewards
    avg_rewards = []

In [ ]:
df_rewards

In [ ]:
greedy_list = ["e-greedy: 0.01", "e-greedy: 0.05", "e-greedy: 0.1", "e-greedy: 0.2"]

df_rewards[greedy_list].iplot(
    title="e-Greedy Actions", dash=["solid", "dash", "dashdot", "dot"]
)

We can see from the above plot that those strategies with the lowest $\epsilon$ values perform the worst early on, as they have very little capacity for exploration and so must accumulate many samples before they can change strategy. However, in the long run, while the higher $\epsilon$ values plateau, we can see the smaller values continuue to increase. This lends itself to the idea that a beneficial strategy would be to start with a large value for early exploration, and then dynamically decrease the value with increasing samples.

#### Disadvantages
* $\epsilon$-greedy actions and A/B/n tests are equally inefficient and static in allocating the exploration budget. In this particular example, you would want to drop the campaigns that are clearly performing extremely poorly and use the exploration budget on the more promising options.
* Modifying the $\epsilon$ greedy approach introduces more hyperparameters that require tuning.

#### Advantages
* Unlike the A/B/n approach, exploration is continuous and therefore it could feasibly adapt to a dynamic environment.
* The $\epsilon$ greedy approach can be made better by dynamically adjusting the value of $\epsilon$
* The approach can be made more dynamic by increasing the importance of the more recent observations. In the update equation for the average reward $Q_{n+1}$, we could replace the factor 1/n with a constant $\alpha$ that allows more recent observations to have greater contribution.


### 2.3 Strategy 3: Upper confidence bounds

UCB is an exploration-exploitation strategy that at each timestep, will choose the action that has the highest potential for reward, where the potential is defined as the sum of the action value estimate and a measure of the uncertainty estimate.

At time t, we can take action $$A_t = \arg\max_a \left[Q_t(a) + c \sqrt{\frac{\ln t}{N_t(a)}} \right]$$

This allows us to either pick an action because it has a high estimated action value (first term) or the uncertainty in the action value is high (second term).

As $\ln t$ grows with time, if the corresponding number of times an action has been taken has not increased accordingly, then it becomes more probable to take that action. The factor $c$ is a hyperparameter that allows us to control the rate of exploration and exploitation.

The second term is derived from Hoeffding's inequality, which bounds the probability that the empirical mean overstimates the actual mean by some error $\epsilon$ for any IID variables drawn from a finite bounded interval, $$P(\bar{X} - \mu \leq \epsilon) \leq e^{-2n\epsilon^2}$$. If we say that the probability should be bounded by some small number $\delta$, then we can solve for:
$$P(\bar{X} - \mu \leq \epsilon) \leq e^{-2n\epsilon^2} \leq \delta$$

which gives us $$\epsilon \geq \sqrt{\frac{-\ln \delta}{2n}}$$

Defining $\delta = 1/t$ so that the bound decreases with time (representing a greater exploitation rate with time), we arrive at the formulation seen above.

In [ ]:
n_prod = 100_000
n_ads = len(ads)
ad_indices = np.array(range(n_ads))

In [ ]:
avg_rewards = []
for c in [0.01, 0.05, 0.1, 0.2]:
    Q = np.zeros(n_ads)
    N = np.zeros(n_ads)
    total_reward = 0
    for t in range(1, n_prod + 1):
        if any(N == 0):
            ad_chosen = np.random.choice(ad_indices[N == 0])
        else:
            uncertainty = np.sqrt(np.log(t) / N)
            ad_chosen = np.argmax(Q + (c * uncertainty))

        R = ads[ad_chosen].pull_lever()
        N[ad_chosen] += 1
        Q[ad_chosen] += (1 / N[ad_chosen]) * (R - Q[ad_chosen])
        total_reward += R
        avg_reward_so_far = total_reward / t
        avg_rewards.append(avg_reward_so_far)

    df_rewards[f"UCB, c={c}"] = avg_rewards
    avg_rewards = []

In [ ]:
ucb_list = ["UCB, c=0.01", "UCB, c=0.05", "UCB, c=0.1", "UCB, c=0.2"]
best_reward = df_rewards.loc[t - 1, ucb_list].max()
df_rewards[ucb_list].iplot(
    title=f"Action Selection using UCB. Best avg. reward: {best_reward:.4f}",
    dash=["solid", "dash", "dashdot", "dot"],
    xTitle="Impressions",
    yTitle="Avg. Reward",
)

#### Disadvantages
* UCB can be hard to tune because the parameter $c$ has less of an intuitive value, especially when compared to the $\epsilon$-greedy approach.

#### Advantages
* UCB systematically and dynamically allocates exploration budget to alternative campaigns that require further exploration. If an ad suddenly becomes much more popular in a changing environment, it will slowly adapt to that.
* UCB can be further optimised for dynamic environments - at the cost of more hyperparameters. For example, we could include exponential smoothing on the Q-values to prioritise more recent observations. There are also some more effective estimations of the uncertainty component.

### 2.4 Thompson Posterior Sampling

The goal of the Multi-Armed Bandit problem is to estimate the parameters of the reward distribution of each arm. In the context of marketing, this corresponds to choosing an ad with the greatest click-through rate. This idea of estimating the distribution's parameters fits within the Bayesian inference framework.

In the ad example, for a given ad $k$, observing a click through is a Bernoulli random variable with parameter $\theta_k$, which we are trying to estimate.

Initially, we don't have a reason to believe that the parameter is high or low for a given ad. So it makes sense to assume a uniform distribution over the interval [0,1].

If we dispay the ad $k$ and it results in a click, we use this as a sinal to update the probability distribution for $\theta_k$ so that the expected value shifts towards 1. As we collect more data, the variance in the estimate for the parameter should shrink, and this is our method for balancing exploration vs exploitation.

The method uses a sample from the posterior distribution of the parameter $p(\theta_k | X)$. If the expected value of $\theta_k$ is high, we are likely to get samples closer to 1. If the variance is high because ad $k$ has not been selected a lot, the samples will also have high variance, which allows for exploration. 

At a given timestep, we take a sample from each ad's posterior distribution and select the greatest sample to choose which ad to display.

It is common to choose the $\beta$ distribution as the prior due to its bounding within the region [0,1] and because it is a conjugate distribution to a Bernoulli likelihood, which ensures the posterior is also a Bernoulli distribution:

$$p(\theta_k) = \frac{\Gamma(\alpha_k + \beta_k)}{\Gamma (\alpha_k) \Gamma (\beta_k)} \theta_k^{\alpha_k - 1} (1 - \theta_k)^{\beta_k - 1}$$

Using a value of $\alpha_k = \beta_k = 1$ allows us to implement a uniform prior. We then use the following update rule after observing reward $R_t$ after selecting ad $k$, which gives us the posterior distribution by setting:

$$\alpha_k \to \alpha_k + R_t$$ 
$$\beta_k \to \beta_k + 1-R_t$$

In [ ]:
n_prod = 100_000
n_ads = len(ads)
alphas = np.ones(n_ads)
betas = np.ones(n_ads)
total_reward = 0
avg_rewards = []

In [ ]:
for i in range(n_prod):
    theta_samples = [np.random.beta(alphas[k], betas[k]) for k in range(n_ads)]

    ad_chosen = np.argmax(theta_samples)
    R = ads[ad_chosen].pull_lever()
    alphas[ad_chosen] += R
    betas[ad_chosen] += 1 - R
    total_reward += R
    avg_reward_so_far = total_reward / (i + 1)
    avg_rewards.append(avg_reward_so_far)

df_rewards["Thompson Sampling"] = avg_rewards

In [ ]:
df_rewards["Thompson Sampling"].iplot(
    title=f"Thompson Sampling Avg. Reward: {avg_reward_so_far:.4f}",
    xTitle="Impressions",
    yTitle="Avg Reward",
)

#### Advantages
* Does not require hyperparameter tuning because we assume a uniform prior.
* It has efficient exploration because it dynamically adapts exploration and exploitation of different outcomes, meaning fewer costly mistakes in production due to tuning of hyperparameters